# NAGuide R methods

Setup basic methods and packages used for all methods

- BiocManager could be moved to methods who are installed from BioConductor

In [1]:
# options("install.lock"=FALSE)

packages_base_R <-
  c("BiocManager", "reshape2", "data.table", "readr", "tibble")

install_rpackage  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    install.packages(pkg)
    library(pkg, character.only = TRUE)
  }
  
}

# used in the large imputation function for two packages
install_bioconductor  <- function(pkg) {
  # If not installed, install the package
  if (!require(pkg, character.only = TRUE)) {
    BiocManager::install(pkg)
    library(pkg, character.only = TRUE)
  }
  
}


for (package in packages_base_R) {
  # Check if the package is already installed
  install_rpackage(pkg = package)
}


Loading required package: BiocManager



Bioconductor version '3.18' is out-of-date; the current release version '3.19'
  is available with R version '4.4'; see https://bioconductor.org/install



Loading required package: reshape2



Loading required package: data.table




Attaching package: ‘data.table’




The following objects are masked from ‘package:reshape2’:

    dcast, melt




Loading required package: readr



Loading required package: tibble



setup can be tricky... trying to integrate as much as possible into conda environment

Copied from [NAGuideR's github](https://github.com/wangshisheng/NAguideR/blob/15ec86263d5821990ad39a8d9f378cf4d76b25fb/inst/NAguideRapp/app.R#L1705-L1849) RShiny application. Adapted to run as standalone function in context of the Snakemake workflow.

- `df` and `df1` ?
- seems quite hacky
- code is only slightly adapted from repo to run here, mainly to install packages on the fly

In [2]:
nafunctions <- function(x, method = "zero") {
  df <- df1 <- as.data.frame(x)
  method <- tolower(method)
  if (method == "zero") {
    df[is.na(df)] <- 0
  }
  else if (method == "minimum") {
    df[is.na(df)] <- min(df1, na.rm = TRUE)
  }
  else if (method == "colmedian") {
    install_rpackage('e1071')
    df <- impute(df1, what = "median")
  }
  else if (method == "rowmedian") {
    install_rpackage('e1071')
    dfx <- impute(t(df1), what = "median")
    df <- t(dfx)
  }
  else if (method == "knn_impute") {
    install_bioconductor('impute')
    data_zero1 <-
      impute.knn(as.matrix(df1),
                 k = 10,
                 rowmax = 1,
                 colmax = 1)#rowmax = 0.9, colmax = 0.9
    df <- data_zero1$data
  }
  else if (method == "seqknn") {
    if (!require(SeqKnn)) {
      install.packages("src/R_NAGuideR/SeqKnn_1.0.1.tar.gz",
                       repos = NULL,
                       type = "source")
      library(SeqKnn)
    }
    df <- SeqKNN(df1, k = 10)
  }
  else if (method == "bpca") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(
        as.matrix(df1),
        nPcs = ncol(df1) - 1,
        method = "bpca",
        maxSteps = 100
      )
    df <- completeObs(data_zero1)
  }
  else if (method == "svdmethod") {
    install_bioconductor('pcaMethods')
    data_zero1 <-
      pcaMethods::pca(as.matrix(df1),
                      nPcs = ncol(df1) - 1,
                      method = "svdImpute")
    df <- completeObs(data_zero1)
  }
  else if (method == "lls") {
    install_bioconductor('pcaMethods')
    data_zero1 <- llsImpute(t(df1), k = 10)
    df <- t(completeObs(data_zero1))
  }
  else if (method == "mle") {
    install_rpackage('norm')
    xxm <- as.matrix(df1)
    ss <- norm::prelim.norm(xxm)
    thx <- norm::em.norm(ss)
    norm::rngseed(123)
    df <- norm::imp.norm(ss, thx, xxm)
  }
  else if (method == "qrilc") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('gmm')
    install_rpackage('imputeLCMD')
    xxm <- t(df1)
    data_zero1 <-
      imputeLCMD::impute.QRILC(xxm, tune.sigma = 1)[[1]]
    df <- t(data_zero1)
  }
  else if (method == "mindet") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('gmm')
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <- imputeLCMD::impute.MinDet(xxm, q = 0.01)
  }
  else if (method == "minprob") {
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('gmm')
    install_rpackage('imputeLCMD')
    xxm <- as.matrix(df1)
    df <-
      imputeLCMD::impute.MinProb(xxm, q = 0.01, tune.sigma = 1)
  }
  else if (method == "irm") {
    install_rpackage('VIM')
    df <- irmi(df1, trace = TRUE, imp_var = FALSE)
    rownames(df) <- rownames(df1)
  }
  else if (method == "impseq") {
    install_rpackage('rrcovNA')
    df <- impSeq(df1)
  }
  else if (method == "impseqrob") {
    install_rpackage('rrcovNA')
    data_zero1 <- impSeqRob(df1, alpha = 0.9)
    df <- data_zero1$x
  }
  else if (method == "mice-norm") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "norm")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "mice-cart") {
    install_rpackage('mice')
    minum <- 5
    datareadmi <- mice(df1,
                       m = minum,
                       seed = 1234,
                       method = "cart")
    newdatareadmi <- 0
    for (i in 1:minum) {
      newdatareadmi <- complete(datareadmi, action = i) + newdatareadmi
    }
    df <- newdatareadmi / minum
    rownames(df) <- rownames(df1)
  }
  else if (method == "trknn") {
    source('src/R_NAGuideR/Imput_funcs.r')
    # sim_trKNN_wrapper <- function(data) {
    #   result <- data %>% as.matrix %>% t %>% imputeKNN(., k=10, distance='truncation', perc=0) %>% t
    #   return(result)
    # }
    # df1x <- sim_trKNN_wrapper(t(df1))
    # df<-as.data.frame(t(df1x))
    df <-
      imputeKNN(as.matrix(df),
                k = 10,
                distance = 'truncation',
                perc = 0)
    df <- as.data.frame(df)
  }
  else if (method == "rf") {
    install_rpackage("missForest")
    data_zero1 <- missForest(
      t(df1),
      maxiter = 10,
      ntree = 20 # input$rfntrees
      ,
      mtry = floor(nrow(df1) ^ (1 / 3)),
      verbose = TRUE
    )
    df <- t(data_zero1$ximp)
  }
  else if (method == "pi") {
    width <- 0.3 # input$piwidth
    downshift <- 1.8 # input$pidownshift
    for (i in 1:ncol(df1)) {
      temp <- df1[[i]]
      if (sum(is.na(temp)) > 0) {
        temp.sd <- width * sd(temp[!is.na(temp)], na.rm = TRUE)
        temp.mean <-
          mean(temp[!is.na(temp)], na.rm = TRUE) - downshift * sd(temp[!is.na(temp)], na.rm = TRUE)
        n.missing <- sum(is.na(temp))
        temp[is.na(temp)] <-
          rnorm(n.missing, mean = temp.mean, sd = temp.sd)
        df[[i]] <- temp
      }
    }
    df
  }
  # else if(method=="grr"){
  #   library(DreamAI)
  #   df<-impute.RegImpute(data=as.matrix(df1), fillmethod = "row_mean", maxiter_RegImpute = 10,conv_nrmse = 1e-03)
  # }
  else if (method == "gms") {
    # install.packages('GMSimpute')
    if (!require(GMSimpute)) {
      install.packages(
        "src/R_NAGuideR/GMSimpute_0.0.1.1.tar.gz",
        repos = NULL,
        type = "source"
      )
      
      library(GMSimpute)
    }
    
    df <- GMS.Lasso(df1,
                    nfolds = 3,
                    log.scale = FALSE,
                    TS.Lasso = TRUE)
  }
  else if (method == "msimpute") {
    install_bioconductor("msImpute")
    df <- msImpute(as.matrix(df),
                   method = 'v2')
    df <- as.data.frame(df)
  }
  else if (method == "msimpute_mnar") {
    install_bioconductor("msImpute")
    df <-
      msImpute(as.matrix(df),
               method = 'v2-mnar',
               group = rep(1, dim(df)[2]))
    df <- as.data.frame(df)
  }
  else if (method == "gsimp") {
    options(stringsAsFactors = F)
    # dependencies parly for sourced file
    
    install_bioconductor("impute")
    install_bioconductor("pcaMethods")
    install_rpackage('gmm')
    install_rpackage('imputeLCMD')
    install_rpackage("magrittr")
    install_rpackage("glmnet")
    install_rpackage("abind")
    install_rpackage("foreach")
    install_rpackage("doParallel")
    source('src/R_NAGuideR/GSimp.R')
    
    # wrapper function with data pre-processing
    pre_processing_GS_wrapper <- function(data_raw_log) {
      # samples in rows, features in columns #
      # Initialization #
      data_raw_log_qrilc <- as.data.frame(data_raw_log) %>%
        impute.QRILC() %>% extract2(1)
      # Centralization and scaling #
      data_raw_log_qrilc_sc <-
        scale_recover(data_raw_log_qrilc, method = 'scale')
      # Data after centralization and scaling #
      data_raw_log_qrilc_sc_df <- data_raw_log_qrilc_sc[[1]]
      # Parameters for centralization and scaling (for scaling recovery) #
      data_raw_log_qrilc_sc_df_param <- data_raw_log_qrilc_sc[[2]]
      # NA position #
      NA_pos <- which(is.na(data_raw_log), arr.ind = T)
      # NA introduced to log-scaled-initialized data #
      data_raw_log_sc <- data_raw_log_qrilc_sc_df
      data_raw_log_sc[NA_pos] <- NA
      # Feed initialized and missing data into GSimp imputation #
      result <-
        data_raw_log_sc %>% GS_impute(
          .,
          iters_each = 50,
          iters_all = 10,
          initial = data_raw_log_qrilc_sc_df,
          lo = -Inf,
          hi = 'min',
          n_cores = 1,
          imp_model = 'glmnet_pred'
        )
      data_imp_log_sc <- result$data_imp
      # Data recovery #
      data_imp <- data_imp_log_sc %>%
        scale_recover(., method = 'recover',
                      param_df = data_raw_log_qrilc_sc_df_param) %>%
        extract2(1)
      return(data_imp)
    }
    df <- t(df) # samples in rows, feature in columns
    df <- pre_processing_GS_wrapper(df)
    df <- t(df) # features in rows, samples in columns
    
  }
  else{
    stop(paste("Unspported methods so far: ", method))
  }
  df <- as.data.frame(df)
  df
}

## Parameters

Choose one of the available methods. 
Some methods might fail for your dataset for unknown reasons
(and the error won't always be easy to understand)
```method
method = 'ZERO'
method = 'MINIMUM'
method = 'COLMEDIAN'
method = 'ROWMEDIAN'
method = 'KNN_IMPUTE'
method = 'SEQKNN'
method = 'BPCA'
method = 'SVDMETHOD'
method = 'LLS'
method = 'MLE'
mehtod = 'LLS'
method = 'QRILC'
method = 'MINDET'
method = 'MINPROB'
method = 'IRM'
method = 'IMPSEQ'
method = 'IMPSEQROB'
method = 'MICE-NORM'
method = 'MICE-CART'
method = 'RF'
method = 'PI'
method = 'GMS'
method = 'TRKNN',
method = 'MSIMPUTE'
method = 'MSIMPUTE_MNAR'
method = 'GSIMP'
```

In [3]:
train_split = 'runs/example/data/data_wide_sample_cols.csv' # test
folder_experiment = 'runs/example/'
method = 'KNN_IMPUTE'

In [4]:
# Parameters
train_split = "runs/alzheimer_study/data/data_wide_sample_cols.csv"
method = "IRM"
folder_experiment = "runs/alzheimer_study"


## Dump predictions

In [5]:
df <-
  utils::read.csv(
    train_split,
    row.names = 1,
    header = TRUE,
    stringsAsFactors = FALSE
  )
df

,Sample_000,Sample_001,Sample_002,Sample_003,Sample_004,Sample_005,Sample_006,Sample_007,Sample_008,Sample_009,⋯,Sample_200,Sample_201,Sample_202,Sample_203,Sample_204,Sample_205,Sample_206,Sample_207,Sample_208,Sample_209
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A0A024QZX5;A0A087X1N8;P35237,15.91237,NA,16.11149,16.10697,15.60321,15.81161,15.49966,15.22075,15.98013,NA,⋯,NA,15.86264,15.65594,15.40126,NA,15.68225,15.79804,15.73949,15.47682,NA
A0A024R0T9;K7ER74;P02655,16.85194,16.87369,NA,17.03151,15.33051,18.61407,17.40917,17.68391,16.38625,16.58972,⋯,17.30955,16.61480,17.95339,18.19938,17.27911,16.88635,17.55400,NA,16.77948,17.26137
A0A024R3W6;A0A024R412;O60462;O60462-2;O60462-3;O60462-4;O60462-5;Q7LBX6;X5D2Q8,15.57047,15.51892,15.93532,15.80187,15.37522,15.62420,15.91185,15.38499,15.89447,15.37538,⋯,NA,15.93179,14.85871,15.12451,NA,14.91006,15.59966,15.46896,14.99496,15.17487
A0A024R644;A0A0A0MRU5;A0A1B0GWI2;O75503,16.48108,16.38694,16.41577,16.97862,16.67906,15.95764,16.23389,16.41793,16.27134,16.20970,⋯,15.95402,16.02959,16.49753,16.51284,16.51269,16.48232,15.93782,16.89773,16.13246,16.23505
A0A075B6H7,17.30138,NA,18.17480,15.96322,NA,18.31720,NA,17.21393,17.79403,NA,⋯,NA,16.48262,14.83467,13.56893,14.48257,NA,NA,NA,NA,NA
A0A075B6H9,20.24646,19.94110,19.25090,19.62797,20.44983,18.68569,20.34722,20.79604,18.99523,19.24667,⋯,NA,NA,18.26986,19.80062,20.18315,17.70485,18.15450,18.63631,14.90754,17.89344
A0A075B6I0,16.76446,18.78624,16.83217,17.85209,18.68185,17.86543,18.24995,20.30936,17.97843,19.41047,⋯,18.00550,18.54142,17.80748,19.89865,19.67420,17.03945,18.15179,17.95040,NA,17.74443
A0A075B6I1,17.58357,17.14388,15.67073,18.87749,17.08131,14.82369,18.51969,19.35640,18.56449,17.96310,⋯,16.41402,17.50510,14.01911,17.50574,20.25111,NA,16.50267,16.32073,NA,16.37054
A0A075B6I6,16.98778,NA,17.01183,14.18153,14.14021,17.04179,17.37560,16.99794,17.20311,16.75839,⋯,15.72543,17.06758,16.45849,16.32778,16.33372,16.41304,16.86003,16.40071,16.11868,15.77958


- `data.frame` does not allow abritary column names, but only valid column names...
- tibbles don't support rownames, and the imputation methods rely on normal `data.frame`s.
Save the header row for later use.

In [6]:
original_header <- colnames(readr::read_csv(
  train_split,
  n_max = 1,
  col_names = TRUE,
  skip = 0
))
feat_name <- original_header[1]
original_header[1:5]

Rows: 1 Columns: 211


── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): protein groups
dbl (180): Sample_000, Sample_002, Sample_003, Sample_004, Sample_005, Sampl...
lgl  (30): Sample_001, Sample_009, Sample_012, Sample_015, Sample_017, Sampl...



ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "protein groups" "Sample_000"     "Sample_001"     "Sample_002"    
[5] "Sample_003"

Uncomment to test certain methods (only for debugging, as at least one method per package is tested using Github Actions)

In [7]:
# to_test <- c(
# 'ZERO',
# 'MINIMUM',
# 'COLMEDIAN',
# 'ROWMEDIAN',
# 'KNN_IMPUTE',
# 'SEQKNN',
# 'BPCA',
# 'SVDMETHOD',
# 'LLS',
# 'MLE',
# 'LLS',
# 'QRILC',
# 'MINDET',
# 'MINPROB',
# 'IRM',
# 'IMPSEQ',
# 'IMPSEQROB',
# 'MICE-NORM',
# 'MICE-CART',
# 'RF',
# 'PI',
# 'GMS', # fails to install on Windows
# 'TRKNN',
# 'MSIMPUTE'
# 'MSIMPUTE_MNAR'
# 'GSIMP'
# )

# for (method in to_test) {
#     print(method)
#     pred <- nafunctions(df, method)
# }

Impute and save predictions with original feature and column names

In [8]:
pred <- nafunctions(df, method)
pred <- tibble::as_tibble(cbind(rownames(pred), pred))
names(pred) <- original_header
pred

Loading required package: VIM



Loading required package: colorspace



Loading required package: grid



VIM is ready to use.




Suggestions and bug-reports can be submitted at: https://github.com/statistikat/VIM/issues



...

[1] "inner loop: 1"
[1] "numeric"
[1] "numeric"
[1] "formula used: Sample_000 ~ Sample_001+Sample_002+Sample_003+Sample_004+Sample_005+Sample_006+Sample_007+Sample_008+Sample_009+Sample_010+Sample_011+Sample_012+Sample_013+Sample_014+Sample_015+Sample_016+Sample_017+Sample_018+Sample_019+Sample_020+Sample_021+Sample_022+Sample_023+Sample_024+Sample_025+Sample_026+Sample_027+Sample_028+Sample_029+Sample_030+Sample_031+Sample_032+Sample_033+Sample_034+Sample_035+Sample_036+Sample_037+Sample_038+Sample_039+Sample_040+Sample_041+Sample_042+Sample_043+Sample_044+Sample_045+Sample_046+Sample_047+Sample_048+Sample_049+Sample_050+Sample_051+Sample_052+Sample_053+Sample_054+Sample_055+Sample_056+Sample_057+Sample_058+Sample_059+Sample_060+Sample_061+Sample_062+Sample_063+Sample_064+Sample_065+Sample_066+Sample_067+Sample_068+Sample_069+Sample_070+Sample_071+Sample_072+Sample_073+Sample_074+Sample_075+Sample_076+Sample_077+Sample_078+Sample_079+Sample_080+Sample_081+Sample_082+Sample_083+Sample_

[1] "it = 4 ,  Wert = 0.484589792010469"
[1] "eps 5"
[1] "test: FALSE"
[1] "0.484589792010469 < 5 = eps"
[1] "      --> finished after 4 iterations"


Imputation performed on the following data set:




           type      #missing
Sample_000 "numeric" "341"   
Sample_001 "numeric" "312"   
Sample_002 "numeric" "288"   
Sample_003 "numeric" "320"   
Sample_004 "numeric" "304"   
Sample_005 "numeric" "277"   
Sample_006 "numeric" "375"   
Sample_007 "numeric" "504"   
Sample_008 "numeric" "330"   
Sample_009 "numeric" "352"   
Sample_010 "numeric" "320"   
Sample_011 "numeric" "307"   
Sample_012 "numeric" "362"   
Sample_013 "numeric" "432"   
Sample_014 "numeric" "524"   
Sample_015 "numeric" "528"   
Sample_016 "numeric" "424"   
Sample_017 "numeric" "414"   
Sample_018 "numeric" "274"   
Sample_019 "numeric" "396"   
Sample_020 "numeric" "340"   
Sample_021 "numeric" "352"   
Sample_022 "numeric" "339"   
Sample_023 "numeric" "364"   
Sample_024 "numeric" "357"   
Sample_025 "numeric" "602"   
Sample_026 "numeric" "440"   
Sample_027 "numeric" "388"   
Sample_028 "numeric" "375"   
Sample_029 "numeric" "352"   
Sample_030 "numeric" "335"   
Sample_031 "numeric" "413"   
Sample_032

protein groups,Sample_000,Sample_001,Sample_002,Sample_003,Sample_004,Sample_005,Sample_006,Sample_007,Sample_008,⋯,Sample_200,Sample_201,Sample_202,Sample_203,Sample_204,Sample_205,Sample_206,Sample_207,Sample_208,Sample_209
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
A0A024QZX5;A0A087X1N8;P35237,15.91237,16.04117,16.11149,16.10697,15.60321,15.81161,15.49966,15.22075,15.98013,⋯,14.77157,15.86264,15.65594,15.40126,14.79869,15.68225,15.79804,15.73949,15.47682,15.15797
A0A024R0T9;K7ER74;P02655,16.85194,16.87369,16.10182,17.03151,15.33051,18.61407,17.40917,17.68391,16.38625,⋯,17.30955,16.61480,17.95339,18.19938,17.27911,16.88635,17.55400,17.56262,16.77948,17.26137
A0A024R3W6;A0A024R412;O60462;O60462-2;O60462-3;O60462-4;O60462-5;Q7LBX6;X5D2Q8,15.57047,15.51892,15.93532,15.80187,15.37522,15.62420,15.91185,15.38499,15.89447,⋯,14.98494,15.93179,14.85871,15.12451,15.58720,14.91006,15.59966,15.46896,14.99496,15.17487
A0A024R644;A0A0A0MRU5;A0A1B0GWI2;O75503,16.48108,16.38694,16.41577,16.97862,16.67906,15.95764,16.23389,16.41793,16.27134,⋯,15.95402,16.02959,16.49753,16.51284,16.51269,16.48232,15.93782,16.89773,16.13246,16.23505
A0A075B6H7,17.30138,14.29715,18.17480,15.96322,16.25849,18.31720,15.61971,17.21393,17.79403,⋯,16.22812,16.48262,14.83467,13.56893,14.48257,15.21015,15.62411,14.67747,15.75713,15.16871
A0A075B6H9,20.24646,19.94110,19.25090,19.62797,20.44983,18.68569,20.34722,20.79604,18.99523,⋯,18.68796,19.88502,18.26986,19.80062,20.18315,17.70485,18.15450,18.63631,14.90754,17.89344
A0A075B6I0,16.76446,18.78624,16.83217,17.85209,18.68185,17.86543,18.24995,20.30936,17.97843,⋯,18.00550,18.54142,17.80748,19.89865,19.67420,17.03945,18.15179,17.95040,17.69260,17.74443
A0A075B6I1,17.58357,17.14388,15.67073,18.87749,17.08131,14.82369,18.51969,19.35640,18.56449,⋯,16.41402,17.50510,14.01911,17.50574,20.25111,17.06869,16.50267,16.32073,17.25035,16.37054
A0A075B6I6,16.98778,16.74000,17.01183,14.18153,14.14021,17.04179,17.37560,16.99794,17.20311,⋯,15.72543,17.06758,16.45849,16.32778,16.33372,16.41304,16.86003,16.40071,16.11868,15.77958


Transform predictions to long format

In [9]:
pred <- reshape2::melt(pred, id.vars = feat_name)
names(pred) <- c(feat_name, 'Sample ID', method)
pred <- pred[reshape2::melt(is.na(df))['value'] == TRUE, ]
pred

,protein groups,Sample ID,IRM
,<chr>,<fct>,<dbl>
11,A0A075B6J9,Sample_000,16.09945
14,A0A075B6P5;P01615,Sample_000,17.89503
15,A0A075B6Q5,Sample_000,16.25281
16,A0A075B6R2,Sample_000,17.01623
19,A0A075B6S5,Sample_000,16.09702
24,A0A087WSY4,Sample_000,16.29080
34,A0A087WU43;A0A087WX17;A0A087WXI5;P12830;P12830-2,Sample_000,16.24710
39,A0A087WW87;A0A087X0Q4;P01614,Sample_000,15.51599
40,A0A087WWA5,Sample_000,14.56224


Check dimension of long format dataframe

In [10]:
dim(pred)

[1] 71601     3

Save predictions to disk

In [11]:
fname = file.path(folder_experiment,
                  'preds',
                  paste0('pred_all_', toupper(method), '.csv'))
write_csv(pred, path = fname)
fname

Warning message:
“The `path` argument of `write_csv()` is deprecated as of readr 1.4.0.
ℹ Please use the `file` argument instead.”


[1] "runs/alzheimer_study/preds/pred_all_IRM.csv"